## Requirements

- pytorch
- torchtext
- pandas
- scikit-learn
- numpy
- tqdm
- gensim


In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torchtext
from torch.utils.data import DataLoader, Dataset, TensorDataset
import pandas as pd 

## Model 설명

- embedding layer
- |
- convolutional layer (kernel = 3 x embedding dim)
- |
- leakyrelu
- |
- dropout
- |
- maxpool w.r.t time axis
- |
- fcn1 for each labels
- | | | | | | |
- fcn2 for each labels ( -> binary output )
- | | | | | | |
- CrossEntropyLoss

In [2]:
class Net(nn.Module):
    def __init__(self, 
                 vocab_size,
                 embedding_dim,
                 len_sentence,
                 channel_size=4,
                 fc_dim=128,
                 padding_idx=1,
                 dropout=0.3,
                 num_labels=7,
                 batch_size=32,
                 is_cuda=False
                ):
        super(Net, self).__init__()
        self.embedding = nn.Embedding(vocab_size+2, embedding_dim=embedding_dim, padding_idx=padding_idx)
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        self.channel_size = channel_size
        self.len_sentence = len_sentence
        self.batch_size = batch_size
        
        self.conv2d = nn.Conv2d(1, out_channels=channel_size, kernel_size=(3, embedding_dim), stride=1)
        # output : batch x channel x (len_sentence - 2) x 1
        # -> squeeze : batch x channel x (len_sentence - 2)
        self.relu = nn.LeakyReLU()
        self.dropout1d = nn.Dropout(p=dropout)
        self.pool1d = nn.MaxPool1d(kernel_size=2)
        # output : batch x channel x (len_sentence - 2) / 2
        
        self.bottleneck_size = channel_size * (len_sentence - 2) / 2
#         print ("Linear size : %sx(%s-2)/2"%(channel_size, len_sentence), self.bottleneck_size)
        assert self.bottleneck_size.is_integer()
        self.bottleneck_size = int(self.bottleneck_size)
        
        self.fcns1 = [nn.Linear(self.bottleneck_size, fc_dim) for i in range(num_labels)]
        self.fcns2 = [nn.Linear(fc_dim, 2) for i in range(num_labels)]
        
        for i, fcn in enumerate(self.fcns1):
            self.add_module("fcn1-"+str(i), fcn)
        
        for i, fcn2 in enumerate(self.fcns2):
            self.add_module("fcn2-"+str(i), fcn2)
        
        self.fc_dim = fc_dim
        self.num_labels = num_labels
    
    def forward(self, sentence, other_features=None):
#         print("sentence ", sentence.shape)
        image = self.embedding(sentence)
#         print(bottleneck.shape)
        image.unsqueeze_(1)
#         print("image ", image.shape)
        
        bottleneck = self.conv2d(image)
        bottleneck.squeeze_(3)
        bottleneck = self.relu(bottleneck)
        bottleneck = self.dropout1d(bottleneck)
        bottleneck = self.pool1d(bottleneck)
#         print("bt shape ", bottleneck.shape)
        
        bottleneck = bottleneck.view(-1, self.bottleneck_size)
        fcns_1 = []
        for i in range(self.num_labels):
            fcns_1.append(self.fcns1[i](bottleneck))
        
        fcns_2 = []
        for i in range(self.num_labels):
            fcns_2.append(self.fcns2[i](fcns_1[i]))
            
        return fcns_2 # return num_labels


In [3]:
class config:
    vocab_size = 20000
    embedding_dim = 50
    len_sentence = 30
    num_labels = 7
    min_freq = 3

In [4]:
def get_pd_data(path : str):
    df = pd.read_csv(path)
    return df

In [5]:
train = get_pd_data('./data/train.csv')

In [6]:
test = get_pd_data('./data/test.csv')

In [7]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


## Preprocess (1)
----
###  Set captial character ratio (not be used now)
- 문장 내의 대문자 비율을 나중에 뉴럴넷의 input으로 줄 예정

In [8]:
def set_capital_ratio(df : pd.DataFrame):
    df['alphas'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isalpha()))
    df['capitals'] = df['comment_text'].apply(lambda comment: sum(1 for c in comment if c.isupper()))
    df['cap_ratio'] = df.apply(lambda row: float(row['capitals']) / (float(row['alphas']) + 1), axis=1)


In [9]:
set_capital_ratio(train), set_capital_ratio(test)

(None, None)

In [10]:
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,alphas,capitals,cap_ratio
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0,203,17,0.083333
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0,73,8,0.108108
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0,186,4,0.021390
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0,486,11,0.022587
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0,50,2,0.039216


## Preprocess(2)
-----
### Word tokenize
- gensim의 tokenize function

In [11]:
from gensim.utils import simple_tokenize

In [12]:
def tokenizer(string : str):
    return [s for s in simple_tokenize(string)]

In [13]:
tk_train = train['comment_text'].str.lower().apply(tokenizer)
tk_test = train['comment_text'].str.lower().apply(tokenizer)

In [14]:
tk_train[:5]

0    [explanation, why, the, edits, made, under, my...
1    [d, aww, he, matches, this, background, colour...
2    [hey, man, i, m, really, not, trying, to, edit...
3    [more, i, can, t, make, any, real, suggestions...
4    [you, sir, are, my, hero, any, chance, you, re...
Name: comment_text, dtype: object

In [15]:
labels = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

## Preprocess(3)
----
### Add Normal column label
- toxic하지 않은 label로 분류되는 것에, normal=1 의 새로운 라벨 추가

In [16]:
train['normal'] = 0
train.loc[train[labels].sum(axis=1) == 0, 'normal'] = 1

In [17]:
labels.append('normal')

In [18]:
y_labels = train[labels]
y_labels.head()

,toxic,severe_toxic,obscene,threat,insult,identity_hate,normal
0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,1
2,0,0,0,0,0,0,1
3,0,0,0,0,0,0,1
4,0,0,0,0,0,0,1


## Validation
----
### 10000 개의 Validation set
    TODO
    Validation 나누기 전에 shuffle

In [19]:
valid_num = 10000

In [20]:
tk_valid = tk_train[-valid_num:]
y_valid = y_labels[-valid_num:]
tk_train = tk_train[:-valid_num]
y_labels = y_labels[:-valid_num]

In [21]:
from torchtext import data, datasets

## Preprocess(3)
---
### torchtext.data.Field
- word dictionary, word to index 구현

In [22]:
TEXT = data.Field(sequential=True,  
                  # 들어갈 데이터가 sequential 인가요? 우리는 tokenize한 word의 sequence를 다룰거니까 True입니다. Defualt로도 True임.
                  tokenize=tokenizer, 
                  # 그 데이터를 tokenize할 함수를 지정할 수 있습니다. 우리는 gensim library의 tokenize 함수를 쓸건데요
                  # 뭐 굳이 그거 말고도 직접 정의해도 되고 str.split 같은걸 써넣어도 됩니다.
                  # :: 그런 줄 알았는데 아무 tokenize 함수나 쓰면 안되고, generator가 아닌 tokenized list 를 반환하는 함수여야합니다..
                  # :::: 이게 아닐거같기도 함.
                  fix_length=config.len_sentence,
                 # 아마 tokenize된 길이 제한 같은데 한번 확인해볼게요. 특이사항으로는 length 넘으면 자르고, 안넘으면 padding을 채웁니다
                  # :: 그게 아니고 vector화 했을 때의 길이 제한일 것 같아요. 확인해보겠습니다.
                  pad_first=True,
                  # padding이 앞에서부터 붙냐, 뒤에서부터 붙냐는 겁니다.
                  tensor_type=torch.cuda.LongTensor
                  # cuda를 써도 됩니다
                 )

In [23]:
TEXT.build_vocab(tk_train, tk_valid, max_size=config.vocab_size, min_freq=config.min_freq)

In [24]:
def batchify(tk_train, y_labels, batch_size=32):
    for i in range(0, len(tk_train), batch_size):
        yield tk_train[i:min(i+batch_size, len(tk_train))], y_labels[i:min(i+batch_size, len(tk_train))]

In [25]:
net = Net(vocab_size=config.vocab_size, embedding_dim=config.embedding_dim, len_sentence=config.len_sentence,
         channel_size=8, num_labels=config.num_labels, batch_size=32).cuda()

In [26]:
net

Net(
  (embedding): Embedding(20002, 50, padding_idx=1)
  (conv2d): Conv2d (1, 8, kernel_size=(3, 50), stride=(1, 1))
  (relu): LeakyReLU(0.01)
  (dropout1d): Dropout(p=0.3)
  (pool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcn1-0): Linear(in_features=112, out_features=128)
  (fcn1-1): Linear(in_features=112, out_features=128)
  (fcn1-2): Linear(in_features=112, out_features=128)
  (fcn1-3): Linear(in_features=112, out_features=128)
  (fcn1-4): Linear(in_features=112, out_features=128)
  (fcn1-5): Linear(in_features=112, out_features=128)
  (fcn1-6): Linear(in_features=112, out_features=128)
  (fcn2-0): Linear(in_features=128, out_features=2)
  (fcn2-1): Linear(in_features=128, out_features=2)
  (fcn2-2): Linear(in_features=128, out_features=2)
  (fcn2-3): Linear(in_features=128, out_features=2)
  (fcn2-4): Linear(in_features=128, out_features=2)
  (fcn2-5): Linear(in_features=128, out_features=2)
  (fcn2-6): Linear(in_features=128, out_features=

In [27]:
optimizer = optim.Adam(net.parameters())
criterions = [nn.CrossEntropyLoss() for i in range(config.num_labels)]

In [28]:
from tqdm import tqdm

In [29]:
net.train(True)

Net(
  (embedding): Embedding(20002, 50, padding_idx=1)
  (conv2d): Conv2d (1, 8, kernel_size=(3, 50), stride=(1, 1))
  (relu): LeakyReLU(0.01)
  (dropout1d): Dropout(p=0.3)
  (pool1d): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fcn1-0): Linear(in_features=112, out_features=128)
  (fcn1-1): Linear(in_features=112, out_features=128)
  (fcn1-2): Linear(in_features=112, out_features=128)
  (fcn1-3): Linear(in_features=112, out_features=128)
  (fcn1-4): Linear(in_features=112, out_features=128)
  (fcn1-5): Linear(in_features=112, out_features=128)
  (fcn1-6): Linear(in_features=112, out_features=128)
  (fcn2-0): Linear(in_features=128, out_features=2)
  (fcn2-1): Linear(in_features=128, out_features=2)
  (fcn2-2): Linear(in_features=128, out_features=2)
  (fcn2-3): Linear(in_features=128, out_features=2)
  (fcn2-4): Linear(in_features=128, out_features=2)
  (fcn2-5): Linear(in_features=128, out_features=2)
  (fcn2-6): Linear(in_features=128, out_features=

In [30]:
def validation(net, tk_valid, y_valid : pd.DataFrame, TEXT : data.Field, criterions : list):
    net.train(False)
    val_corrects = [0 for i in range(config.num_labels)]
    val_score = []
    valid_loss = 0 
    softmax = nn.Softmax(dim=1)
    for val_step, (batch_val, y_val) in enumerate(batchify(tk_valid, y_valid.values)):
        y_total_loss = 0
        var_batch = TEXT.process(batch_val, device=0, train=False)
        var_y = Variable(torch.cuda.LongTensor(y_val)).transpose(dim0=0, dim1=1)
        pred_score = net(var_batch.transpose(dim0=0, dim1=1))
        val_score.append([softmax(score).data.cpu() for score in pred_score])
        for i, score in enumerate(pred_score):
            _, pred = score.max(dim=1)
            val_corrects[i] += (pred == var_y[i]).float().sum()
            y_loss = criterions[i](score, var_y[i])
            y_total_loss += y_loss
        valid_loss += y_total_loss
    net.train(True)
    valid_loss /= len(tk_valid) / 32
    
    for i, val_correct in enumerate(val_corrects):
        val_corrects[i] = val_corrects[i] / len(tk_valid)
        
    return valid_loss, val_corrects, val_score

In [38]:
train_corrects = [0 for i in range(config.num_labels)]
train_loss = 0

for step, (batch, y_label) in tqdm(enumerate(batchify(tk_train, y_labels.values))):
    
    var_batch = TEXT.process(batch, device=0, train=True)
    var_y = Variable(torch.cuda.LongTensor(y_label)).transpose(dim0=0, dim1=1)

    pred_score = net(var_batch.transpose(dim0=0, dim1=1))
    
    net.zero_grad()
    y_total_loss = 0
    for i, score in enumerate(pred_score):
        _, pred = score.max(dim=1)
        train_corrects[i] += (pred == var_y[i]).float().sum()
        y_loss = criterions[i](score, var_y[i])
#         print(y_loss.data[0])
        y_total_loss += y_loss
    
    y_total_loss.backward()
    if step % 1000 == 999:
        valid_loss, valid_acc, val_score = validation(net, tk_valid, y_valid, TEXT, criterions)
        print("valid loss", valid_loss)
        print("valid acc", [i.data[0] for i in valid_acc])

    optimizer.step()
    

1015it [00:08, 114.43it/s]

valid loss Variable containing:
 0.5868
[torch.cuda.FloatTensor of size 1 (GPU 0)]

valid acc [0.9477999806404114, 0.9908999800682068, 0.9709999561309814, 0.9960999488830566, 0.9642999768257141, 0.9905999898910522, 0.946899950504303]


2024it [00:17, 114.18it/s]

valid loss Variable containing:
 0.5916
[torch.cuda.FloatTensor of size 1 (GPU 0)]

valid acc [0.949999988079071, 0.991599977016449, 0.9716999530792236, 0.9966999888420105, 0.9666999578475952, 0.9909999966621399, 0.9496999979019165]


3020it [00:26, 114.05it/s]

valid loss Variable containing:
 0.6037
[torch.cuda.FloatTensor of size 1 (GPU 0)]

valid acc [0.9503999948501587, 0.9914000034332275, 0.9731999635696411, 0.9966999888420105, 0.9679999947547913, 0.9908999800682068, 0.9496999979019165]


4017it [00:35, 114.04it/s]

valid loss Variable containing:
 0.5763
[torch.cuda.FloatTensor of size 1 (GPU 0)]

valid acc [0.9494999647140503, 0.9914000034332275, 0.9722999930381775, 0.9966999888420105, 0.9657999873161316, 0.9908999800682068, 0.9498999714851379]


4675it [00:40, 116.82it/s]


In [39]:
valid_loss, valid_acc, val_score = validation(net, tk_valid, y_valid, TEXT, criterions)

In [40]:
predictions = [] # 모든 라벨의 validation set에 대한 예측값
for i in range(config.num_labels):
    prediction = [s[i] for s in val_score] # i번째 라벨에 대한 예측값을 배치에 따라 모음
    predictions.append(torch.cat(prediction))


In [41]:
from sklearn.metrics import roc_auc_score


## TODO
---
### roc_auc_score w.r.t. validation set's score
- Kaggle form에 맞추어 column-wise roc auc score 계산

In [42]:
import numpy as np

In [43]:
roc_auc_scores = 0
for i in range(config.num_labels - 1): # minus 1 for normal 
    score = roc_auc_score( y_valid[labels[i]].values, predictions[i].numpy()[:, 1])
    print(score)
    roc_auc_scores += score

0.941990985132
0.977937669675
0.963982988188
0.936143820061
0.95841659655
0.898814889562


In [44]:
print(roc_auc_scores / (config.num_labels - 1))

0.946214491528
